In [2]:
import pandas as pd
from data_processing.encoding.categorical import CategoricalTransformer
from data_processing.encoding.numeric_and_date import ToQuantileTransformer
from data_processing.encoding.text2vec import TextTransform

Load relevant data

In [3]:
data = pd.read_csv('data/other_data/PreFer_fake_data.csv')
codebook = pd.read_csv('data/codebooks/PreFer_codebook.csv')
summary = pd.read_csv('data/codebooks/PreFer_codebook_summary.csv')

Get the two relevant columns (Only core questions for now)

In [4]:
categorical_columns = codebook[(codebook.var_name.str.startswith('c')) & (codebook.type_var == 'categorical')].var_name.tolist()
quantile_columns = codebook[(codebook.var_name.str.startswith('c')) & ((codebook.type_var == 'numeric') | (codebook.type_var == 'date or time'))].var_name.tolist()

In [5]:
cat_transform = CategoricalTransformer()
cat_transform.fit(codebook)

/Users/lmmi/fertility-prediction-challenge/data_processing/encoding/categorical.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  core_cat_df['values_cat'] = core_cat_df['values_cat'].str.split("; ").apply(lambda x: [e.strip() for e in x])
/Users/lmmi/fertility-prediction-challenge/data_processing/encoding/categorical.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  core_cat_df['labels_cat'] = core_cat_df['labels_cat'].str.split("; ").apply(lambda x: [e.strip() for e in x])


This can be changed to work on a dataframe

In [6]:
for col in categorical_columns:
    data[col] = cat_transform.transform(data[col])

In [7]:
quantile_transform = ToQuantileTransformer(quantile_columns)
quantile_transform.fit(data)
data = quantile_transform.transform(data)

Replace nans and set all columns to int

In [8]:
data.fillna(101, inplace=True)
data[quantile_columns] = data[quantile_columns].astype(int)

In [9]:
data2 = data[data.columns[data.columns.str.startswith('c')]]
data2['nomem_encr'] = data['nomem_encr']

/var/folders/y6/j9fbqcvx6lb5l99614n30y4c0000gn/T/ipykernel_5158/1456275911.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data2['nomem_encr'] = data['nomem_encr']
/var/folders/y6/j9fbqcvx6lb5l99614n30y4c0000gn/T/ipykernel_5158/1456275911.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['nomem_encr'] = data['nomem_encr']


In [10]:
from data_processing.sequences.sequencing import to_sequences

sequences = to_sequences(data2, summary)

In [15]:
import numpy as np
index = np.array([isinstance(x, str) for x in sequences[700001]['08']])
np.array(sequences[700001]['08'])[index]

array(['long answer', 'answer', 'answ', 'answ', 'answer', 'answer',
       'answer', 'answ', 'long answer', 'answ', 'answer', 'answer',
       'answ', 'answer', 'answ', 'answer', 'answ', 'answ', 'answer',
       'answer', 'answ', 'answ', 'answer', 'answ', 'answ', 'answer',
       'even longer answer', 'answer', 'answer', 'answer', 'answ',
       'answer', 'answ', 'answer', 'answer', 'answer', 'long answer',
       'answ', 'answer', 'long answer', 'answ', 'answ', 'answ', 'answer',
       'even longer answer', 'answ', 'answer', 'answer', 'answer',
       'even longer answer', 'answer', 'answer', 'answ', 'answer', 'answ',
       'answer', 'answer', 'long answer', 'answer', 'answ',
       'even longer answer', 'answer', 'answ', 'answer', 'answer',
       'answer', 'long answer', 'answer', 'answ', 'answ', 'answ',
       'answer', 'even longer answer', 'answer', 'answer'], dtype='<U32')